# Travel time estimation using tensor decomposition

Resources:

https://www.microsoft.com/en-us/research/publication/trajectory-data-mining-an-overview/ 

https://www.microsoft.com/en-us/research/wp-content/uploads/2015/09/TrajectoryDataMining-tist-yuzheng.pdf

https://towardsdatascience.com/recommendation-system-matrix-factorization-d61978660b4b

Formula to implement:


\begin{aligned}
\mathcal{L}(S, R, U, T, F, G)=&\frac{1}{2}\left\|\mathcal{A}-S \times_R R \times_U U \times_T T\right\|^2+\frac{\lambda_1}{2}\|X-T G\|^2 \\
+&\frac{\lambda_2}{2}\|Y-R F\|^2+\frac{\lambda_3}{2}\left(\|S\|^2+\|R\|^2+\|U\|^2+\|T\|^2+\|F\|^2+\|G\|^2\right)
\end{aligned}




\begin{aligned}
\phi_{i j k}= & S \times_R R_{i *} \times_U U_{j *} \times_T T_{k *} ; \\
R_{i *} & \leftarrow R_{i *}-\eta \lambda_3 R_{i *}-\eta\left(\phi_{i j k}-\mathcal{A}_{i j k}\right) \times S \times_U U_{j *} \times_T T_{k *} \\
& -\eta \lambda_2\left(R_{i *} \times F-Y_{i *}\right) \times F ; \\
U_{j *} \leftarrow & U_{j *}-\eta \lambda_3 U_{j *}-\eta\left(\phi_{i j k}-\mathcal{A}_{i j k}\right) \times S \times_R R_{i *} \times_T T_{k *} \\
T_{k *} \leftarrow & T_{k *}-\eta \lambda_3 T_{k *}-\eta\left(\phi_{i j k}-\mathcal{A}_{i j k}\right) \times S \times_R R_{i *} \times_U U_{j *} \\
& \quad-\eta \lambda_1\left(T_{k *} \times G-X_{k *}\right) \times G ; \\
S \leftarrow & S-\eta \lambda_3 S-\eta\left(\phi_{i j k}-\mathcal{A}_{i j k}\right) \times R_{i *} \otimes U_{j *} \otimes T_{k *} \\
G \leftarrow & G-\eta \lambda_3 G-\eta \lambda_1\left(T_{k *} \times G-X_{k *}\right) \times T_{k *} \\
F \leftarrow & F-\eta \lambda_3 F-\eta \lambda_2\left(R_{i *} \times F-Y_{i *}\right) \times R_{i *}
\end{aligned}


## Formula explaination

This formula describes the loss function for a tensor decomposition (factorization) method in order to estimate the three-dimensional tensor $\mathcal{A}$. 

$\mathcal{A}$ encodes the user-specific travel time in the following way: and entry $\mathcal{A_{ijk}}$ of $\mathcal{A}$ is the time that the $j$-th driver takes to travel in the $i$-th road segment in the $k$-th daily time slot. Therefore $\mathcal{A}$ is essentially a three dimensional array. We can know certain entries of this tensor by tracking user's GPS signal, but a large amount of entries in this tensor are unknows, as users will only drive in certain roads at certain times, therefore we aim to estimate them. Intuitively, we could try to guess some of the missing entries based on existing ones by eyeballing correlations in the data: for example, if at a given time of the day the travel time for users is large in certain road segments, we expect that there is traffic there, and therefore we can estimate long travel times for other drivers, too.


In order to do this 2 more matrices are introduced: $X$ and $Y$

We need to estimate the tensor A by decomposing it into a product. The measure of the "fitness" of the decomposition is given by the sum of the square of the difference between the entries for which A is known and its decomposition, ie.e the Froenbius norm. From this, we compute the gradient of this loss function with respect each component of the matrices we decompose A into, and use this in a gradient descent algorithm. Each iteration computes the gradient and updates the matrices in such a way the loss is minimized.

## Implementation

In [2]:
import numpy as np

In [3]:
def phi_tensor(s,r,u,t):
    ''' phi tensor, defined above.
    it's the product of the tensor s with the matrices r,u,t
    has the same dimensions of A'''
    return np.einsum('ijk,ai,bj,ck->abc',s,r,u,t)
def phi_ijk(s,r,u,t,a,b,c):
    '''(i,j,k)th component of phi,
    i.e. the contraction of s with r,u,t tensors'''
    return np.einsum('ijk,i,j,k->',s,r[a,:],u[b,:],t[c,:])
def phi_ijk_2(s,ri,uj,tk):
    '''a possibly more optimized version
    ri,uj,tk=r[i,:],u[j,:],t[k,:]'''
    return np.einsum('ijk,i,j,k->',s,ri,uj,tk)
def rut(r,u,t,a,b,c):
    '''tensor product of i,j,k th components
    of r,u,t'''
    return np.einsum('i,j,k->ijk',r[a,:],u[b,:],t[c,:])


In [4]:
t1=np.ones(shape=(2,2))
a2D = np.array([[1, 2], [3, 4]])
sel=np.array([[1, 0], [1, 0]])
np.einsum('ij,ij,ji->',t1,a2D,sel)

3.0

In [5]:
np.tensordot(t1,a2D,axes=0)

array([[[[1., 2.],
         [3., 4.]],

        [[1., 2.],
         [3., 4.]]],


       [[[1., 2.],
         [3., 4.]],

        [[1., 2.],
         [3., 4.]]]])

In [6]:
#latent dimensions
dU=5
dR=7
dT=3
#
dX=11 #aka p
dY=13 #aka q
#
n=3 #road segments
m= 9 #drivers
l= 4#time slots


In [7]:
#tensor initialization

s=np.random.rand(dR,dU,dT)
r=np.random.rand(n,dR)
u=np.random.rand(m,dU)
t=np.random.rand(2*l,dT)
phi=np.einsum('ijk,ai,bj,ck->abc',s,r,u,t)#tensor contraction between s,r,u,t: should be a nxmx2l tensor
f=np.random.rand(dR,dY)
g=np.random.rand(dT,dX)

In [8]:
## those tensors whould be computed from data, here I randomly initialize them

a=np.random.randint(0,15,size=(n,m,2*l))
x=np.random.randint(0,5,size=(2*l,dX))
y=np.random.randint(0,4,size=(n,dY))

print(a.shape)

(3, 9, 8)


In [9]:
np.where(a > 0, 1, 0)#select only entries where a is greater than 0

array([[[1, 1, 1, 1, 1, 1, 1, 0],
        [0, 1, 0, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 0, 1, 1],
        [0, 1, 1, 1, 1, 0, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 0, 1, 1, 1],
        [1, 1, 0, 1, 1, 1, 1, 1],
        [1, 1, 0, 1, 1, 1, 1, 1]],

       [[1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 1],
        [0, 1, 1, 1, 1, 1, 0, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1]],

       [[1, 1, 1, 0, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 0, 1, 1, 1]]])

In [10]:
c=np.einsum('ai,bj,ck->aibjck',r,u,t)
c.shape

(3, 7, 9, 5, 8, 3)

In [11]:
phi.shape

(3, 9, 8)

In [12]:
i,j,k=0,1,5
phi[i][j][k]

8.703700905514776

In [13]:
t.shape

(8, 3)

In [14]:
g.shape

(3, 11)

In [20]:
(t@g-x)[k,:].shape

(11,)

In [21]:
t.T[:,k]

array([0.78950115, 0.43060236, 0.01633793])

In [24]:
np.einsum('a,b->ab',t.T[:,k],(t@g-x)[k,:])

array([[-2.91371357e+00, -2.98270532e+00, -2.74896665e+00,
         8.04947745e-01, -1.28885761e+00, -4.19519264e-02,
        -3.93418715e-01, -5.87644734e-01, -4.37893046e-01,
        -1.21319615e+00, -2.63166116e+00],
       [-1.58917048e+00, -1.62679932e+00, -1.49931576e+00,
         4.39027093e-01, -7.02956700e-01, -2.28810285e-02,
        -2.14574767e-01, -3.20507711e-01, -2.38831542e-01,
        -6.61690133e-01, -1.43533609e+00],
       [-6.02963625e-02, -6.17240772e-02, -5.68870914e-02,
         1.66575814e-02, -2.66716078e-02, -8.68152788e-04,
        -8.14140337e-03, -1.21607149e-02, -9.06175478e-03,
        -2.51058703e-02, -5.44595724e-02]])

In [27]:
np.tensordot( t[k,:],t[k,:]@g-x[k,:],axes=0)

array([[-2.91371357e+00, -2.98270532e+00, -2.74896665e+00,
         8.04947745e-01, -1.28885761e+00, -4.19519264e-02,
        -3.93418715e-01, -5.87644734e-01, -4.37893046e-01,
        -1.21319615e+00, -2.63166116e+00],
       [-1.58917048e+00, -1.62679932e+00, -1.49931576e+00,
         4.39027093e-01, -7.02956700e-01, -2.28810285e-02,
        -2.14574767e-01, -3.20507711e-01, -2.38831542e-01,
        -6.61690133e-01, -1.43533609e+00],
       [-6.02963625e-02, -6.17240772e-02, -5.68870914e-02,
         1.66575814e-02, -2.66716078e-02, -8.68152788e-04,
        -8.14140337e-03, -1.21607149e-02, -9.06175478e-03,
        -2.51058703e-02, -5.44595724e-02]])

In [160]:
np.einsum('ijk,i,j,k->',s,r[i,:],u[j,:],t[k,:])

4.496344530819598

In [161]:
print(rut(r,u,t,0,0,0).shape)

(7, 5, 3)


In [163]:
np.einsum('abc,b,c->a',s,u[j],t[k]).shape

(7,)

In [164]:
r[i,:]-(phi_ijk(s,r,u,t,i,j,k)-a[i,j,k])*np.einsum('abc,b,c->a',s,u[j],t[k])-(r[i]@f-y[i])@f.T

array([ 1.66454547,  1.13463229,  0.48652242,  1.22359074, -1.50237014,
        1.34449524,  1.78321477])

In [165]:
def matrix_factorization(a,x,y,epsilon=0.001,steps=1000,eta=0.001,dR=10,dU=10,dT=10,lamb1=1,lamb2=1,lamb3=1):
    '''
    element-wise optimization algorithm
    a: target tensor of shape n x m x l 
    x,y: context matrices
    steps: max iterations
    eta: learning rate
    epsilon: treshold for early stop
    dR,dU,dT: dimensions of latent spaces
    '''
    n,m,twol=a.shape
    dG=x.shape[1]
    dF=y.shape[1]
    #initialize s,r,u,t,f,g
    s=np.random.rand(dR,dU,dT)
    r=np.random.rand(n,dR)
    u=np.random.rand(m,dU)
    t=np.random.rand(twol,dT)
    f=np.random.rand(dR,dF)
    g=np.random.rand(dT,dG)
    for steps in range(steps):
        if True :#loss_i-loss_i-1 > epsilon
            for i in range(n):
                for j in range(m):
                    for k in range(twol):
                        if a[i,j,k]>0:
                            phiijk=phi_ijk(s,r,u,t,i,j,k)
                            #compute all gradients
                            gradr=lamb3* r[i,:]+(phi_ijk(s,r,u,t,i,j,k)-a[i,j,k])*np.einsum('abc,b,c->a',s,u[j],t[k])+lamb2*(r[i]@f-y[i])@f.T
                            gradu=lamb3* u[j,:]+(phi_ijk(s,r,u,t,i,j,k)-a[i,j,k])*np.einsum('abc,a,c->b',s,r[i],t[k])
                            gradt=lamb3* t[k,:]+(phi_ijk(s,r,u,t,i,j,k)-a[i,j,k])*np.einsum('abc,a,b->c',s,r[i],u[j])+lamb1*(t[k]@g-x[k])@g.T
                            grads=lamb3*s+ (phi_ijk(s,r,u,t,i,j,k)-a[i,j,k])*rut(r,u,t,i,j,k)
                            gradg=lamb3*g+lamb1*np.tensordot( t[k,:],t[k,:]@g-x[k,:],axes=0)
                            gradf=lamb3*f+lamb2*np.tensordot( r[i,:],r[i,:]@f-y[i,:],axes=0)
                            #update all parameters
                            r[i]=r[i] -eta*gradr
                            u[j]=u[j]- eta*gradu
                            t[k]=t[k]- eta*gradt
                            g=g-gradg*eta
                            f=f-gradf*eta
                            s=s-eta*grads
        #for i,j,k for which a[ijk]=! 0
            #compute gradient of 
    return s,r,u,t

SyntaxError: invalid syntax (428390435.py, line 33)